# STAT4012 Group Project
> The file is used to test the code for the project.  
> It should be written with clear comments and explanations on each sections of the code.  
> If there are some other modules that need to be imported or run in this file, you can use %load filename.py to load the code from the file. use %run filename.py to run the code from the file.  

In [10]:
# import libraries
# all the libraries are imported here but not below
import numpy as np
import pandas as pd


## Import Raw Data

In [14]:
raw_data_adjusted = pd.read_excel(
    '../data/STAT4012_fundamental_technical_raw_data_check_normal_abnormal_cash_dividend.xlsx', engine='openpyxl')
raw_data_unadjusted = pd.read_excel(
    '../data/STAT4012_fundamental_technical_raw_data.xlsx', engine='openpyxl')
